In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from functools import reduce

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import seaborn as sns

import statsmodels.api as sm
from linearmodels.panel import PanelOLS
from scipy.stats import ttest_ind

from utils import *

spark = get_spark_session()
sns.set(font_scale=2, style='white')

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

In [ ]:
with open('config.json', 'r') as infile:
    config = json.load(infile)
indir = config['indir']
outdir = config['outdir']
admin_fname = config['admin_fname']
canton_assignment_fname = config['cantons_assignment_fname']
rct_assignment_fname = config['rct_assignment_fname']
shapefile_fname = config['shapefile_fname']

### I. Plot unique MSISDN and IMEIs by day, remove outliers

In [ ]:
toplevel_dir = outdir + 'january/'
months = ['2021-01']

cols = ['caller_msisdn', 'operator', 'month', 'day', 'imei']
voice = spark.read.csv(indir + 'voice/2021/1.csv', header=True)\
    .select(cols)\
    .where(col('month').isin(months))\
    .na.drop(subset=['caller_msisdn', 'month', 'day', 'imei'])
sms = spark.read.csv(indir + 'sms/2021/1.csv', header=True)\
    .select(cols)\
    .where(col('month').isin(months))\
    .na.drop(subset=['caller_msisdn', 'month', 'day', 'imei'])
cdr = voice.union(sms)

In [ ]:
msisdn_by_day = cdr.groupby('day').agg(countDistinct('caller_msisdn'))\
    .withColumnRenamed('count(caller_msisdn)', 'count')
save_df(msisdn_by_day, toplevel_dir + 'msisdn_by_day.csv')

imei_by_day = cdr.groupby('day').agg(countDistinct('imei'))\
    .withColumnRenamed('count(imei)', 'count')
save_df(imei_by_day, toplevel_dir + 'imei_by_day.csv')

In [ ]:
msisdn_by_day = pd.read_csv(toplevel_dir + 'msisdn_by_day.csv')
msisdn_by_day['day'] = pd.to_datetime(msisdn_by_day['day'])
msisdn_by_day = msisdn_by_day.sort_values('day', ascending=True)

imei_by_day = pd.read_csv(toplevel_dir + 'imei_by_day.csv')
imei_by_day['day'] = pd.to_datetime(imei_by_day['day'])
imei_by_day = imei_by_day.sort_values('day', ascending=True)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 5))

ax[0].scatter(msisdn_by_day['day'], msisdn_by_day['count']/1000000, color='indianred')
ax[0].plot(msisdn_by_day['day'], msisdn_by_day['count']/1000000, color='indianred')
ax[0].set_title('MSISDNs by Day')
ax[0].set_ylabel('MSISDNs (millions)')
msisdn_sd_below = msisdn_by_day['count'].mean() - 2*msisdn_by_day['count'].std()
msisdn_sd_above = msisdn_by_day['count'].mean() + 2*msisdn_by_day['count'].std()
ax[0].axhline(msisdn_sd_above/1000000, color='grey', dashes=[1, 1])
ax[0].axhline(msisdn_sd_below/1000000, color='grey', dashes=[1, 1])

ax[1].scatter(imei_by_day['day'], imei_by_day['count']/1000000, color='mediumseagreen')
ax[1].plot(imei_by_day['day'], imei_by_day['count']/1000000, color='mediumseagreen')
ax[1].set_title('IMEIs by Day')
ax[1].set_ylabel('IMEIs (millions)')
imei_sd_below = imei_by_day['count'].mean() - 2*imei_by_day['count'].std()
imei_sd_above = imei_by_day['count'].mean() + 2*imei_by_day['count'].std()
ax[1].axhline(imei_sd_below/1000000, color='grey', dashes=[1, 1])
ax[1].axhline(imei_sd_above/1000000, color='grey', dashes=[1, 1])

locator = mdates.DayLocator(interval=7)
formatter = mdates.DateFormatter('%m-%d')
for a in range(len(ax)):
    ax[a].xaxis.set_major_locator(locator)
    ax[a].xaxis.set_major_formatter(formatter)
    simpleaxis(ax[a])

plt.show()

outliers_msisdn = list(msisdn_by_day[(msisdn_by_day['count'] < msisdn_sd_below) | \
                                     (msisdn_by_day['count'] > msisdn_sd_above)]['day'])
outliers_imei = list(msisdn_by_day[(msisdn_by_day['count'] < msisdn_sd_below) | \
                                   (msisdn_by_day['count'] > msisdn_sd_above)]['day'])
outliers = list(set([x.strftime('%Y-%m-%d') for x in outliers_msisdn + outliers_imei]))
pd.DataFrame(outliers, columns=['outliers']).to_csv(toplevel_dir + 'outliers.csv', index=False)

In [ ]:
cdr = cdr.where(~col('day').isin(outliers))

imei_per_msisdn = cdr.groupby('caller_msisdn').agg(countDistinct('imei'))\
    .withColumnRenamed('count(imei)', 'count')
save_df(imei_per_msisdn, toplevel_dir + 'imei_per_msisdn.csv')

msisdn_per_imei = cdr.groupby('imei').agg(countDistinct('caller_msisdn'))\
    .withColumnRenamed('count(caller_msisdn)', 'count')
save_df(msisdn_per_imei, toplevel_dir + 'msisdn_per_imei.csv')

count_txns_msisdn = cdr.groupby('caller_msisdn').agg(count('imei'))\
    .withColumnRenamed('count(imei)', 'count')
save_df(count_txns_msisdn, toplevel_dir + 'count_transactions_msisdn.csv')

count_txns_imei = cdr.groupby('imei').agg(count('caller_msisdn'))\
    .withColumnRenamed('count(caller_msisdn)', 'count')
save_df(count_txns_imei, toplevel_dir + 'count_transactions_imei.csv')

### II. Generate datasets of IMEI/MSISD and MSISDN/IMEI

In [ ]:
toplevel_dir = outdir + 'october'
months = ['2020-10']

outliers = list(pd.read_csv(toplevel_dir + 'outliers.csv')['outliers'])
cols = ['caller_msisdn', 'operator', 'month', 'day', 'imei', 'recipient_msisdn']
voice = spark.read.csv(indir + 'voice/2020/11.csv', header=True)\
    .select(cols)\
    .where(col('month').isin(months))\
    .na.drop(subset=['caller_msisdn', 'month', 'day', 'imei'])\
    .where(~(col('day').isin(outliers)))
sms = spark.read.csv(indir + 'sms/2020/11.csv', header=True)\
    .select(cols)\
    .where(col('month').isin(months))\
    .na.drop(subset=['caller_msisdn', 'month', 'day', 'imei'])\
    .where(~(col('day').isin(outliers)))
cdr = voice.union(sms)

In [ ]:
imei_per_msisdn = cdr.groupby('caller_msisdn').agg(count('imei'))
msisdn_per_imei = cdr.groupby('imei').agg(count('caller_msisdn'))
save_df(imei_per_msisdn, toplevel_dir + '/imei_per_msisdn.csv')
save_df(msisdn_per_imei, toplevel_dir + '/msisdn_per_imei.csv')

### Define treatment at IMEI level

In [ ]:
cols = ['caller_msisdn', 'day', 'imei', 'recipient_msisdn', 'interaction']

voice_oct = spark.read.csv(indir + 'voice/2020/10.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])
sms_oct = spark.read.csv(indir + 'sms/2020/10.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])
           
voice_nov = spark.read.csv(indir + 'voice/2020/11.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])
sms_nov = spark.read.csv(indir + 'sms/2020/11.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])

voice_dec = spark.read.csv(indir + 'voice/2020/12.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])
sms_dec = spark.read.csv(indir + 'cdr/sms/2020/12.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])

voice_jan = spark.read.csv(indir + 'voice/2021/1.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])
sms_jan = spark.read.csv(indir + 'sms/2021/1.csv', header=True)\
    .select(cols)\
    .na.drop(subset=['caller_msisdn', 'day', 'imei'])

cdr = reduce(DataFrame.unionAll, [voice_nov, sms_nov, voice_dec, sms_dec])

In [ ]:
novissi = pd.read_csv(admin_fname)\
    [['Gender', 'Age', 'Prefecture', 'Canton', 'RegistrationDate', 'PhoneNumber']]
novissi['RegistrationDate'] = pd.to_datetime(novissi['RegistrationDate'])
novissi = novissi[novissi['RegistrationDate'] <= pd.to_datetime('2020-12-31')]
novissi['rural'] = novissi['Prefecture'].apply(lambda x: 0 if x in ['GOLFE', 'AGOE-NYIVE', 'TCHAOUDJO'] else 1)

# Check registered November-December
novissi['registered_nov_dec'] = ((novissi['RegistrationDate'] >= pd.to_datetime('2020-11-01')) & \
                                 (novissi['RegistrationDate'] <= pd.to_datetime('2020-12-31')))\
                                .astype('int')
novissi['registered_nov'] = ((novissi['RegistrationDate'] >= pd.to_datetime('2020-11-01')) & \
                             (novissi['RegistrationDate'] <= pd.to_datetime('2020-11-30')))\
.astype('int')
print('Percent between November and December: %.2f' % (100*(novissi['registered_nov_dec'].mean())))

# Check in targeted canton
targeted_cantons = pd.read_csv(cantons_assignment_fname)\
    .rename({'novissi_prefecture_name':'Prefecture', 'novissi_canton_name':'Canton', 
             'targeted':'targeted_canton'}, axis=1)\
    [['targeted_canton', 'Prefecture', 'Canton']]
novissi = novissi.merge(targeted_cantons, on=['Prefecture', 'Canton'], how='left')
novissi['targeted_canton'] = novissi['targeted_canton'].fillna(0)
print('Percent in targeted cantons: %.2f' % (100*(novissi[novissi['registered_nov_dec'] == 1]['targeted_canton']\
                                                  .mean())))

# Check below wealth threshold
wealth_scores = pd.read_csv(rct_assignment_fname)\
    .rename({'phone_number':'PhoneNumber', 'cdr_pmt':'poverty', 'treatment':'treated'}, axis=1)\
    [['PhoneNumber', 'poverty', 'treated']]
novissi = novissi.merge(wealth_scores, on='PhoneNumber', how='left')
novissi['below_wealth_threshold'] = (novissi['treated'] >= 0).astype('int')
print('Percent below wealth threshold: %.2f' % (100*(novissi[(novissi['registered_nov_dec'] == 1) & 
                                                             (novissi['targeted_canton'] == 1)]\
                                                     ['below_wealth_threshold'].mean())))
print('Percent treated: %.2f' % (100*(novissi[(novissi['registered_nov_dec'] == 1) & 
                                              (novissi['targeted_canton'] == 1) & 
                                              (novissi['below_wealth_threshold'] == 1)]\
                                                     ['treated'].mean())))

In [ ]:
admin = spark.createDataFrame(novissi)

In [ ]:
min_time = cdr.withColumn('day', to_timestamp(col('day')))\
    .withColumnRenamed('caller_msisdn', 'PhoneNumber')\
    .join(admin, on=['PhoneNumber'])\
    .withColumn('timedif', abs(datediff(col('day'), col('RegistrationDate'))))\
    .withColumn('min_timedif', min('timedif').over(Window.partitionBy('PhoneNumber')))\
    .where(col('timedif') == col('min_timedif'))

max_count = min_time.groupby(['PhoneNumber', 'imei']).agg(count('timedif'), first('timedif'))\
    .withColumnRenamed('count(timedif)', 'count_txns_on_day')\
    .withColumnRenamed('first(timedif)', 'days_to_txn')\
    .withColumn('max_txns', max('count_txns_on_day').over(Window.partitionBy('PhoneNumber')))\
    .where(col('count_txns_on_day') == col('max_txns'))

random_choice = max_count.withColumn('rand', rand())\
    .withColumn('max_random', max('rand').over(Window.partitionBy('PhoneNumber')))\
    .where(col('rand') == col('max_random'))

In [ ]:
more_than_one_min_imei = min_time.groupby('PhoneNumber').agg(countDistinct('imei'))\
    .where(col('count(imei)') > 1)\
    .count()

print('Share of phone numbers with more than one IMEI on minimum day: %.2f' % 
     (100*more_than_one_min_imei/min_time.select('PhoneNumber').distinct().count()))

In [ ]:
more_than_one_max_count = max_count.groupby('PhoneNumber').agg(countDistinct('imei'))\
    .where(col('count(imei)') > 1)\
    .count()

print('Share of phone numbers with more than one IMEI with the max count on the minimum day: %.2f' % 
     (100*more_than_one_max_count/max_count.select('PhoneNumber').distinct().count()))

In [ ]:
save_df(random_choice, indir + 'imei_assignments.csv')

In [ ]:
imeis = pd.read_csv(indir + 'imei_assignments.csv')
print('Share of transactions within 1 day of registration: %.2f' % 
     (100*len(imeis[imeis['hours_to_txn'] <= 1])/len(imeis)) + '%')
print('Share of transactions within 7 days of registration: %.2f' % 
     (100*len(imeis[imeis['hours_to_txn'] <= 7])/len(imeis)) + '%')
print('Share of transactions within 30 days of registration: %.2f' % 
     (100*len(imeis[imeis['hours_to_txn'] <= 30])/len(imeis)) + '%')
grouped = imeis.groupby('imei').agg('count')
print('Share of IMEIs linked to more than one MSISDN: %.2f' % 
     (100*len(grouped[grouped['PhoneNumber'] > 1])/len(grouped)))

In [ ]:
print('IMEI-centric dataset: %i' % len(imeis))
imeis_deduplicate = imeis.drop_duplicates(subset=['imei'])
imeis_deduplicate.to_csv(outdir + 'imei_assignments_deduplicated.csv',
                        index=False)
print('IMEI-centric dataset, deduplicated: %i' % len(imeis_no_duplicates))
counts = imeis.groupby('imei', as_index=False).agg('count')[['imei', 'PhoneNumber']]
no_duplicates = set(counts[counts['PhoneNumber'] == 1]['imei'])
imeis_drop_duplicates = imeis[imeis['imei'].isin(no_duplicates)]
imeis_drop_duplicates.to_csv(outdir + 'imei_assignments_drop_duplicates.csv',
                        index=False)
print('IMEI-centric dataset, no duplicates: %i' % len(imeis_drop_duplicates))

### III. Table 2 Panel A: Entire Network

In [ ]:
toplevel_dir = indir + 'january/'

imei_per_msisdn = pd.read_csv(toplevel_dir + 'imei_per_msisdn.csv')
msisdn_per_imei = pd.read_csv(toplevel_dir + 'msisdn_per_imei.csv')
outcomes = [('imei_per_msisdn', imei_per_msisdn), 
            ('msisdn_per_imei', msisdn_per_imei)]

In [ ]:
table = []
for label, outcome in outcomes:
    column = ['%.2f (%.2f)' % (outcome['count'].mean(), outcome['count'].std()), 
              '%.2f'% outcome['count'].median(), 
              '%.2f' % outcome['count'].mode()[0], 
              '%.2f' % (100*(outcome['count'] > 1).mean()),
              '%i' % len(outcome)]
    table.append(column)

table = pd.DataFrame(table).T
table.columns = [label for label, _ in outcomes]
table.index = ['Mean', 'Median', 'Mode', 'Share', 'N']
table

### IV. Match to administrative data

In [ ]:
novissi = pd.read_csv(admin_fname)[['Gender', 'Age', 'Prefecture', 'Canton', 'RegistrationDate', 'PhoneNumber']]
novissi['RegistrationDate'] = pd.to_datetime(novissi['RegistrationDate'])
novissi = novissi[novissi['RegistrationDate'] <= pd.to_datetime('2020-12-31')]
novissi['rural'] = novissi['Prefecture'].apply(lambda x: 0 if x in 
                                               ['LOME COMMUNE', 'GOLFE', 'AGOE-NYIVE', 'TCHAOUDJO'] else 1)

# Check registered November-December
novissi['registered_nov_dec'] = ((novissi['RegistrationDate'] >= pd.to_datetime('2020-11-01')) & \
                                 (novissi['RegistrationDate'] <= pd.to_datetime('2020-12-31')))\
                                .astype('int')
novissi['registered_nov'] = ((novissi['RegistrationDate'] >= pd.to_datetime('2020-11-01')) & \
                             (novissi['RegistrationDate'] <= pd.to_datetime('2020-11-30')))\
.astype('int')
print('Percent between November and December: %.2f' % (100*(novissi['registered_nov_dec'].mean())))

# Check in targeted canton
targeted_cantons = pd.read_csv(canton_assignment_fname)\
    .rename({'novissi_prefecture_name':'Prefecture', 'novissi_canton_name':'Canton', 
             'targeted':'targeted_canton'}, axis=1)\
    [['targeted_canton', 'Prefecture', 'Canton']]
novissi = novissi.merge(targeted_cantons, on=['Prefecture', 'Canton'], how='left')
novissi['targeted_canton'] = novissi['targeted_canton'].fillna(0)
print('Percent in targeted cantons: %.2f' % (100*(novissi[novissi['registered_nov_dec'] == 1]['targeted_canton']\
                                                  .mean())))

# Check below wealth threshold
wealth_scores = pd.read_csv(rct_assignment_fname)\
    .rename({'phone_number':'PhoneNumber', 'cdr_pmt':'poverty', 'treatment':'treated'}, axis=1)\
    [['PhoneNumber', 'poverty', 'treated']]
novissi = novissi.merge(wealth_scores, on='PhoneNumber', how='left')
novissi['below_wealth_threshold'] = (novissi['treated'] >= 0).astype('int')
print('Percent below wealth threshold: %.2f' % (100*(novissi[(novissi['registered_nov_dec'] == 1) & 
                                                             (novissi['targeted_canton'] == 1)]\
                                                     ['below_wealth_threshold'].mean())))
print('Percent treated: %.2f' % (100*(novissi[(novissi['registered_nov_dec'] == 1) & 
                                              (novissi['targeted_canton'] == 1) & 
                                              (novissi['below_wealth_threshold'] == 1)]\
                                                     ['treated'].mean())))

# Link to IMEI
imeis = pd.read_csv(indir + 'scratch/imei_assignments_deduplicated.csv')\
    [['PhoneNumber', 'imei']]
novissi = novissi.merge(imeis, on='PhoneNumber', how='left')
print('Percent of records linked to IMEI: %.2f' % (100 - 100*novissi['imei'].isnull().mean()))

### Table 1 Panel A and Table 2 Panel B: Matched to Admin Data

In [ ]:
df = novissi[novissi['registered_nov_dec'] == 1].copy()
print(df.shape)

imei_per_msisdn = pd.read_csv(toplevel_dir + 'imei_per_msisdn.csv')\
    .rename({'caller_msisdn':'PhoneNumber', 'count':'imei_per_msisdn'}, axis=1)
msisdn_per_imei = pd.read_csv(toplevel_dir + 'msisdn_per_imei.csv')\
    .rename({'count':'msisdn_per_imei'}, axis=1)
counts_msisdn = pd.read_csv(toplevel_dir + 'count_transactions_msisdn.csv')\
    .rename({'caller_msisdn':'PhoneNumber', 'count':'msisdn_count_txns'}, axis=1)
counts_imei = pd.read_csv(toplevel_dir + 'count_transactions_imei.csv')\
    .rename({ 'count':'imei_count_txns'}, axis=1)

msisdn_data = imei_per_msisdn.drop_duplicates(subset=['PhoneNumber'])\
    .merge(counts_msisdn.drop_duplicates(subset=['PhoneNumber']), on='PhoneNumber', how='left')
imei_data = msisdn_per_imei.drop_duplicates(subset=['imei'])\
    .merge(counts_imei.drop_duplicates(subset=['imei']), on='imei', how='left')
           
df = df.merge(msisdn_data, on='PhoneNumber', how='left')\
    .merge(imei_data, on='imei', how='left')

print(df.shape)
print('Percent with MSISDN data: %.2f' % (100 - 100*df['imei_per_msisdn'].isnull().mean()))
print('Percent with IMEI data: %.2f' % (100 - 100*df['msisdn_per_imei'].isnull().mean()))

In [ ]:
table = []
for outcome in ['imei_per_msisdn', 'msisdn_per_imei']:
    df_outcome = df.dropna(subset=[outcome])
    column = ['%.2f (%.2f)' % (df_outcome[outcome].mean(), df_outcome[outcome].std()), 
              '%.2f'% df_outcome[outcome].median(), 
              '%.2f' % df_outcome[outcome].mode()[0], 
              '%.2f' % (100*(df_outcome[outcome] > 1).mean()),
              '%i' % len(df_outcome)]
    table.append(column)

table = pd.DataFrame(table).T
table.columns = ['IMEI per MSISDN', 'MSISDN per IMEI']
table.index = ['Mean', 'Median', 'Mode', 'Share', 'N']
table

In [ ]:
table = []
df_outcome = df.copy()

df_outcome['Age: 18-30'] = df_outcome['Age'].apply(lambda x: x <= 30).astype('int')
df_outcome['Age: 30-40'] = df_outcome['Age'].apply(lambda x: x > 30 and x <= 40).astype('int')
df_outcome['Age: 40-50'] = df_outcome['Age'].apply(lambda x: x > 40 and x <=50).astype('int')
df_outcome['Age: 50+'] = df_outcome['Age'].apply(lambda x: x > 50).astype('int')

column = ['%i' % (len(df_outcome[df_outcome['Gender'] == 'F'])), 
          '%i' % (len(df_outcome[df_outcome['Gender'] == 'M'])),
         '%i' % (len(df_outcome[df_outcome['Age: 18-30'] == 1])),
         '%i' % (len(df_outcome[df_outcome['Age: 30-40'] == 1])),
         '%i' % (len(df_outcome[df_outcome['Age: 40-50'] == 1])),
         '%i' % (len(df_outcome[df_outcome['Age: 50+'] == 1])),
         '%i' % (len(df_outcome[df_outcome['rural'] == 1])),
         '%i' % (len(df_outcome[df_outcome['rural'] == 0]))]
table.append(column)

column = ['%.2f' % (100*len(df_outcome[df_outcome['Gender'] == 'F'])/len(df_outcome)) + '%', 
          '%.2f' % (100*len(df_outcome[df_outcome['Gender'] == 'M'])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 18-30'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 30-40'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 40-50'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 50+'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['rural'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['rural'] == 0])/len(df_outcome)) + '%']
table.append(column)

table = pd.DataFrame(table).T
table.columns = ['N', '%']
table.index = ['Female', 'Male', '18-30', '30-40', '40-50', '50+', 'Rural', 'Urban']
print('N=%i' % len(df_outcome))
table

### Table 1 Panel B and Table 2 Panel C: Enrolled in RCT

In [ ]:
table = []
df_rct = df[(df['registered_nov_dec'] == 1) & (df['targeted_canton'] == 1) & 
            (df['below_wealth_threshold'] == 1)].copy()
for outcome in ['imei_per_msisdn', 'msisdn_per_imei']:
    df_outcome = df_rct.dropna(subset=[outcome])
    column = ['%.2f (%.2f)' % (df_outcome[outcome].mean(), df_outcome[outcome].std()), 
              '%.2f'% df_outcome[outcome].median(), 
              '%.2f' % df_outcome[outcome].mode()[0], 
              '%.2f' % (100*(df_outcome[outcome] > 1).mean()),
              '%i' % len(df_outcome)]
    table.append(column)

table = pd.DataFrame(table).T
table.columns = ['IMEI per MSISDN', 'MSISDN per IMEI']
table.index = ['Mean', 'Median', 'Mode', 'Share', 'N']
table

In [ ]:
table = []

df_outcome = df_rct.copy()

df_outcome['Age: 18-30'] = df_outcome['Age'].apply(lambda x: x <= 30).astype('int')
df_outcome['Age: 30-40'] = df_outcome['Age'].apply(lambda x: x > 30 and x <= 40).astype('int')
df_outcome['Age: 40-50'] = df_outcome['Age'].apply(lambda x: x > 40 and x <=50).astype('int')
df_outcome['Age: 50+'] = df_outcome['Age'].apply(lambda x: x > 50).astype('int')

column = ['%i' % (len(df_outcome[df_outcome['Gender'] == 'F'])), 
          '%i' % (len(df_outcome[df_outcome['Gender'] == 'M'])),
         '%i' % (len(df_outcome[df_outcome['Age: 18-30'] == 1])),
         '%i' % (len(df_outcome[df_outcome['Age: 30-40'] == 1])),
         '%i' % (len(df_outcome[df_outcome['Age: 40-50'] == 1])),
         '%i' % (len(df_outcome[df_outcome['Age: 50+'] == 1])),
         '%i' % (len(df_outcome[df_outcome['rural'] == 1])),
         '%i' % (len(df_outcome[df_outcome['rural'] == 0]))]
table.append(column)

column = ['%.2f' % (100*len(df_outcome[df_outcome['Gender'] == 'F'])/len(df_outcome)) + '%', 
          '%.2f' % (100*len(df_outcome[df_outcome['Gender'] == 'M'])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 18-30'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 30-40'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 40-50'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['Age: 50+'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['rural'] == 1])/len(df_outcome)) + '%',
         '%.2f' % (100*len(df_outcome[df_outcome['rural'] == 0])/len(df_outcome)) + '%']
table.append(column)
table = pd.DataFrame(table).T
table.columns = ['N', '%']
table.index = ['Female', 'Male', '18-30', '30-40', '40-50', '50+', 'Rural', 'Urban']
print(len(df_outcome))
table

### Appendix Table: Baseline Tests for Balance

In [ ]:
table = []
df_balance = df_rct.copy()
df_balance['female'] = df_balance['Gender'].apply(lambda x: 1 if x == 'F' else 0)
df_balance['shared_msisdn'] = df_balance['imei_per_msisdn']\
    .apply(lambda x: 1 if x > 1 else 0 if x == 1 else np.nan)
df_balance['shared_imei'] = df_balance['msisdn_per_imei']\
    .apply(lambda x: 1 if x > 1 else 0 if x == 1 else np.nan)

treatment = df_balance[df_balance['treated'] == 1]
control = df_balance[df_balance['treated'] == 0]

for var in ['female', 'Age', 'shared_msisdn', 'shared_imei', 'msisdn_count_txns']:
    ttest = ttest_ind(treatment[var].dropna(), control[var].dropna()).pvalue 
    stars = '***' if ttest < 0.001 else '**' if ttest < 0.01 else '*' if ttest < 0.05 else ''
    row = ['%.2f (%.2f)' % (treatment[var].mean(), treatment[var].std()), 
          '%.2f (%.2f)' % (control[var].mean(), control[var].std()),
          '%.2f' % (treatment[var].mean() - control[var].mean()) + stars]
    table.append(row)
    
table.append([len(treatment), len(control), len(df_balance)])
table = pd.DataFrame(table)
table.columns = ['Treatment', 'Control', 'Difference']
table.index = ['% Women', 'Age', 'Shared SIM', 'Shared Device', 'Number of Transactions', 'N']
table

### Figure 1: Geographic Heterogeneity

In [ ]:
data = df.copy()

data['imei_per_msisdn'] = 100*(data['imei_per_msisdn'] > 1).astype('int')
data['msisdn_per_imei'] = 100*(data['msisdn_per_imei'] > 1).astype('int')

pop = data.groupby('Prefecture', as_index=False).agg('count')[['Prefecture', 'Gender']]\
    .rename({'Gender':'count'}, axis=1)
pop = pop[pop['count'] > 10]
avg = data.groupby('Prefecture', as_index=False).agg('mean')
avg = pop.merge(avg, on='Prefecture')
shapefile = gpd.read_file(shapefile_fname)[['PREFECTURE', 'geometry']]\
    .rename({'PREFECTURE':'Prefecture'}, axis=1)
shapefile['Prefecture'] = shapefile['Prefecture']\
    .apply(lambda x: 'KPENDJAL OUEST' if x == 'KPENDJAL-OUEST' else 
          'GOLFE' if x == 'LOME COMMUNE' else
          'OTI SUD' if x == 'OTI-SUD' else
          'MO' if x == 'PLAINE DU MO' else x)
avg = shapefile.merge(avg, on='Prefecture', how='inner')

sns.reset_orig()
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
from matplotlib.ticker import FuncFormatter

fmt = lambda x, pos: '{:.1%}'.format(x)
#cbar = plt.colorbar(format=FuncFormatter(fmt))
avg.plot(ax=ax[0], column=['imei_per_msisdn'], legend=True, 
         legend_kwds={'label': 'Percent Shared', 'shrink':0.5}, edgecolor='darkgrey', linewidth=0.5)

shapefile.plot(ax=ax[1], color='lightgrey')
avg.plot(ax=ax[1], column=['msisdn_per_imei'], legend=True,
        legend_kwds={'label': 'Percent Shared', 'shrink':0.5}, edgecolor='darkgrey', linewidth=0.5)

cities = pd.DataFrame([[6.19, 1.2254], [8.9228, 1.1353]])
cities.columns = ['longitude', 'latitude']
cities = gpd.GeoDataFrame(cities, geometry=gpd.points_from_xy(cities['latitude'], cities['longitude']))
cities.plot(ax=ax[0], color='darkred', markersize=200, linewidth=0.5, edgecolor='black')
cities.plot(ax=ax[1], color='darkred', markersize=200, linewidth=0.5, edgecolor='black')

ax[0].axis('off')
ax[1].axis('off')
ax[0].set_title('Shared MSISDNs', fontsize='xx-large')
ax[1].set_title('Shared IMEIs', fontsize='xx-large')

ax[0].annotate('Lomé', xy=[.27, .105], xytext=[.47, .15], xycoords='figure fraction', 
               textcoords='figure fraction', arrowprops={'arrowstyle':'->'}, fontsize='x-large')
ax[1].annotate('Lomé', xy=[.75, .105], xytext=[.47, .15], xycoords='figure fraction', 
               textcoords='figure fraction', arrowprops={'arrowstyle':'->'}, fontsize='x-large')
ax[0].annotate('Tchaoudjo', xy=[.25, .555], xytext=[.35, .82], xycoords='figure fraction', 
               textcoords='figure fraction', arrowprops={'arrowstyle':'->'}, fontsize='x-large')
ax[1].annotate('Tchaoudjo', xy=[.74, .555], xytext=[.35, .82], xycoords='figure fraction', 
               textcoords='figure fraction', arrowprops={'arrowstyle':'->'}, fontsize='x-large')
plt.savefig(outdir + 'maps.png', dpi=600, bbox_inches='tight')
plt.show()

### Table 3: Heterogeneity

In [ ]:
data = df.copy()
data = data[data['registered_nov_dec'] == 1]
data['Gender'] = data['Gender'].apply(lambda x: 1 if x == 'F' else 0)
data['Age: 18-30'] = data['Age'].apply(lambda x: x <= 30).astype('int')
data['Age: 30-40'] = data['Age'].apply(lambda x: x > 30 and x <= 40).astype('int')
data['Age: 40-50'] = data['Age'].apply(lambda x: x > 40 and x <=50).astype('int')
data['Age: 50+'] = data['Age'].apply(lambda x: x > 50).astype('int')

In [ ]:
outcome = 'imei_per_msisdn'
regressions = [
    ('Gender', ['Gender']),
    ('Gender', ['msisdn_count_txns', 'Gender']),
    ('Age', ['Age: 30-40', 'Age: 40-50', 'Age: 50+']),
    ('Age', ['msisdn_count_txns', 'Age: 30-40', 'Age: 40-50', 'Age: 50+']),
    ('Geography', ['rural']),
    ('Geography', ['msisdn_count_txns', 'rural']),
    ('Joint', ['msisdn_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50', 'Age: 50+', 'rural'])
]

table = []
for label, covariates in regressions:
    regression_data = data[covariates + [outcome]].dropna().copy()
    regression_data[outcome] = (regression_data[outcome] > 1).astype('int')

    y = regression_data[outcome]
    x = sm.add_constant(regression_data[covariates].copy())
    result = sm.OLS(y, x).fit()
    
    stars = {result.pvalues.index[i]:'***' if result.pvalues[i] < 0.001 else '**' if result.pvalues[i] < 0.01 \
            else '*' if result.pvalues[i] < 0.05 else '' for i in range(len(result.pvalues))}
    params = {result.params.index[i]: ('%.4f' % result.params[i]) + stars[result.params.index[i]] \
              for i in range(len(result.params))}
    table.append({**params, **{'rsquared': '%.4f' % result.rsquared, 'n':'%i' % result.nobs}})

table = pd.DataFrame(table).T.sort_index().fillna('')
table.columns = [label for label, _ in regressions]
table = table.reindex(index=['const', 'msisdn_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50',
                             'Age: 50+', 'rural', 'rsquared', 'n'])
table

In [ ]:
outcome = 'msisdn_per_imei'
regressions = [
    ('Gender', ['Gender']),
    ('Gender', ['imei_count_txns', 'Gender']),
    ('Age', ['Age: 30-40', 'Age: 40-50', 'Age: 50+']),
    ('Age', ['imei_count_txns', 'Age: 30-40', 'Age: 40-50', 'Age: 50+']),
    ('Geography', ['rural']),
    ('Geography', ['imei_count_txns', 'rural']),
    ('Joint', ['imei_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50', 'Age: 50+', 'rural'])
]

table = []
for label, covariates in regressions:
    regression_data = data[covariates + [outcome]].dropna().copy()
    regression_data[outcome] = (regression_data[outcome] > 1).astype('int')

    y = regression_data[outcome]
    x = sm.add_constant(regression_data[covariates].copy())
    result = sm.OLS(y, x).fit()
    
    stars = {result.pvalues.index[i]:'***' if result.pvalues[i] < 0.001 else '**' if result.pvalues[i] < 0.01 \
            else '*' if result.pvalues[i] < 0.05 else '' for i in range(len(result.pvalues))}
    params = {result.params.index[i]: ('%.4f' % result.params[i]) + stars[result.params.index[i]] \
              for i in range(len(result.params))}
    table.append({**params, **{'rsquared': '%.4f' % result.rsquared, 'n':'%i' % result.nobs}})

table = pd.DataFrame(table).T.sort_index().fillna('')
table.columns = [label for label, _ in regressions]
table = table.reindex(index=['const', 'imei_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50',
                             'Age: 50+', 'rural', 'rsquared', 'n'])
table

### Table 4: Impact Evaluation and Table 6: Heterogeneous Treatment Effects

In [ ]:
data = df.copy()
data = data[data['registered_nov_dec'] == 1]
data = data[data['targeted_canton'] == 1]
data = data[data['below_wealth_threshold'] == 1]
data['Gender'] = data['Gender'].apply(lambda x: 1 if x == 'F' else 0)
data['Age: 18-30'] = data['Age'].apply(lambda x: x <= 30).astype('int')
data['Age: 30-40'] = data['Age'].apply(lambda x: x > 30 and x <= 40).astype('int')
data['Age: 40-50'] = data['Age'].apply(lambda x: x > 40 and x <=50).astype('int')
data['Age: 50+'] = data['Age'].apply(lambda x: x > 50).astype('int')
data['Treatment x Female'] = data['Gender']*data['treated']
data['Treatment x Age: 50+'] = data['Age: 50+']*data['treated']
data['Treatment x Age: 30-40'] = data['Age: 30-40']*data['treated']
data['Treatment x Age: 40-50'] = data['Age: 40-50']*data['treated']

In [ ]:
outcome = 'imei_per_msisdn'
regressions = [
    ('Basic', ['treated']),
    ('Transactions Control', ['treated', 'msisdn_count_txns']),
    ('Demographic controls', ['treated', 'msisdn_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50', 'Age: 50+']),
    ('Gender Heterogeneous Effects', ['treated', 'msisdn_count_txns', 'Gender', 'Treatment x Female']),
    ('Age Heterogeneous Effects', ['treated', 'msisdn_count_txns', 'Age: 30-40', 'Age: 40-50', 'Age: 50+',
                                  'Treatment x Age: 30-40', 'Treatment x Age: 40-50', 'Treatment x Age: 50+'])
]

table = []
for label, covariates in regressions:
    regression_data = data[covariates + [outcome]].dropna().copy()
    regression_data[outcome] = (regression_data[outcome] > 1).astype('int')

    y = regression_data[outcome]
    x = sm.add_constant(regression_data[covariates].copy())
    result = sm.OLS(y, x).fit()
    
    stars = {result.pvalues.index[i]:'***' if result.pvalues[i] < 0.001 else '**' if result.pvalues[i] < 0.01 \
            else '*' if result.pvalues[i] < 0.05 else '' for i in range(len(result.pvalues))}
    params = {result.params.index[i]: ('%.4f' % result.params[i]) + stars[result.params.index[i]] \
              for i in range(len(result.params))}
    table.append({**params, **{'rsquared': '%.4f' % result.rsquared, 'n':'%i' % result.nobs}})

table = pd.DataFrame(table).T.sort_index().fillna('')
table.columns = [label for label, _ in regressions]
table = table.reindex(['const', 'treated', 'msisdn_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50', 
                       'Age: 50+', 'Treatment x Female', 'Treatment x Age: 30-40', 'Treatment x Age: 40-50', 
                       'Treatment x Age: 50+', 'rsquared', 'n'])
table

In [ ]:
outcome = 'msisdn_per_imei'
regressions = [
    ('Basic', ['treated']),
    ('Transactions Control', ['treated', 'imei_count_txns']),
    ('Demographic controls', ['treated', 'imei_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50', 'Age: 50+']),
    ('Gender Heterogeneous Effects', ['treated', 'imei_count_txns', 'Gender', 'Treatment x Female']),
    ('Age Heterogeneous Effects', ['treated', 'imei_count_txns', 'Age: 30-40', 'Age: 40-50', 'Age: 50+',
                                  'Treatment x Age: 30-40', 'Treatment x Age: 40-50', 'Treatment x Age: 50+'])
]

table = []
for label, covariates in regressions:
    regression_data = data[covariates + [outcome]].dropna().copy()
    regression_data[outcome] = (regression_data[outcome] > 1).astype('int')

    y = regression_data[outcome]
    x = sm.add_constant(regression_data[covariates].copy())
    result = sm.OLS(y, x).fit()
    
    stars = {result.pvalues.index[i]:'***' if result.pvalues[i] < 0.001 else '**' if result.pvalues[i] < 0.01 \
            else '*' if result.pvalues[i] < 0.05 else '' for i in range(len(result.pvalues))}
    params = {result.params.index[i]: ('%.4f' % result.params[i]) + stars[result.params.index[i]] \
              for i in range(len(result.params))}
    table.append({**params, **{'rsquared': '%.4f' % result.rsquared, 'n':'%i' % result.nobs}})

table = pd.DataFrame(table).T.sort_index().fillna('')
table.columns = [label for label, _ in regressions]
table = table.reindex(['const', 'treated', 'imei_count_txns', 'Gender', 'Age: 30-40', 'Age: 40-50', 
                       'Age: 50+', 'Treatment x Female', 'Treatment x Age: 30-40', 'Treatment x Age: 40-50', 
                       'Treatment x Age: 50+', 'rsquared', 'n'])
table

### Table 5: Diff-in-diff

In [ ]:
df = novissi.copy()

for month in ['october', 'november', 'december', 'january']:
    toplevel_dir = indir + month + '/'

    imei_per_msisdn = pd.read_csv(toplevel_dir + 'imei_per_msisdn.csv')\
        .rename({'caller_msisdn':'PhoneNumber', 'count':'imei_per_msisdn_' + month}, axis=1)\
        .drop_duplicates(subset=['PhoneNumber'])
    msisdn_per_imei = pd.read_csv(toplevel_dir + 'msisdn_per_imei.csv')\
        .rename({'count':'msisdn_per_imei_' + month}, axis=1)\
        .drop_duplicates(subset=['imei'])
    counts_msisdn = pd.read_csv(toplevel_dir + 'count_transactions_msisdn.csv')\
        .rename({'caller_msisdn':'PhoneNumber', 'count':'msisdn_count_txns_' + month}, axis=1)\
        .drop_duplicates(subset=['PhoneNumber'])
    counts_imei = pd.read_csv(toplevel_dir + 'count_transactions_imei.csv')\
        .rename({'count':'imei_count_txns_' + month}, axis=1)\
        .drop_duplicates(subset=['imei'])
    
    df = df.merge(imei_per_msisdn, on='PhoneNumber', how='left')
    df = df.merge(msisdn_per_imei, on='imei', how='left')
    df = df.merge(counts_msisdn, on='PhoneNumber', how='left')
    df = df.merge(counts_imei, on='imei', how='left')

In [ ]:
data = df.copy()
data = data[data['registered_nov_dec'] == 1]
data = data[data['targeted_canton'] == 1]
data = data[data['below_wealth_threshold'] == 1]

data = data[['PhoneNumber', 'imei', 'treated', 'registered_nov'] + \
            [col for col in df.columns if 'count' in col or 'imei_per_msisdn' in col or 'msisdn_per_imei' in col]]

columns = ['PhoneNumber', 'imei', 'treated', 'imei_per_msisdn', 'msisdn_per_imei', 'msisdn_count_txns', 
          'imei_count_txns', 'month', 'treatmentxmonth']
october = data[['PhoneNumber', 'imei', 'treated', 'registered_nov'] + \
               [col for col in data.columns if 'october' in col]].copy()
october['month'] = 10
october['treatmentxmonth'] = 0
october = october.drop('registered_nov', axis=1)
october.columns = columns

november = data[['PhoneNumber', 'imei', 'treated', 'registered_nov'] + \
                [col for col in data.columns if 'november' in col]].copy()
november['month'] = 11
november['treatmentxmonth'] = november['registered_nov']*november['treated']
november = november.drop('registered_nov', axis=1)
november.columns = columns

december = data[['PhoneNumber', 'imei', 'treated', 'registered_nov'] + \
                [col for col in data.columns if 'december' in col]].copy()
december['month'] = 12
december['treatmentxmonth'] = december['treated']
december = december.drop('registered_nov', axis=1)
december.columns = columns

january = data[['PhoneNumber', 'imei', 'treated', 'registered_nov'] + \
               [col for col in data.columns if 'january' in col]].copy()
january['month'] = 1
january['treatmentxmonth'] = january['treated']
january = january.drop('registered_nov', axis=1)
january.columns = columns

data = pd.concat([october, november, december, january])
data['month_dummy'] = data['month']
data = data.set_index(['PhoneNumber', 'month'])
data = pd.get_dummies(data, columns=['month_dummy'], drop_first=False)
data = data.drop('month_dummy_10', axis=1)

data['msisdn_per_imei'] = data['msisdn_per_imei'].apply(lambda x: np.nan if pd.isnull(x) else 1 if x > 1 else 0)
data['imei_per_msisdn'] = data['imei_per_msisdn'].apply(lambda x: np.nan if pd.isnull(x) else 1 if x > 1 else 0)

In [ ]:
sns.set(font_scale=2)
timeseries = data.copy()
timeseries['month_int'] = [x[1] for x in timeseries.index]
timeseries['month_date'] = timeseries['month_int'].apply(lambda x: '2020-10' if x == 10
                                                        else '2020-11' if x == 11
                                                        else '2020-12' if x == 12
                                                        else '2021-01')
timeseries['month_date'] = pd.to_datetime(timeseries['month_date'])

fig, ax = plt.subplots(1, figsize=(10, 6))

msisdn_timeseries = timeseries[['month_date', 'imei_per_msisdn', 'treated']].dropna().copy()
avg = msisdn_timeseries.groupby(['month_date', 'treated'], as_index=False).agg('mean')
count = msisdn_timeseries.groupby(['month_date', 'treated'], as_index=False).agg('count')
treated = avg[avg['treated'] == 1]
untreated = avg[avg['treated'] == 0]
ax.plot(treated['month_date'], 100*treated['imei_per_msisdn'], label='Shared MSISDNs', 
       color='indianred')
ax.scatter(treated['month_date'], 100*treated['imei_per_msisdn'], color='indianred')
ax.plot(untreated['month_date'], 100*untreated['imei_per_msisdn'], 
        color='indianred', dashes=[2, 2])
ax.scatter(untreated['month_date'], 100*untreated['imei_per_msisdn'], color='indianred')

imei_timeseries = timeseries[['month_date', 'msisdn_per_imei', 'treated', 'imei']].dropna().copy()
avg = imei_timeseries.groupby(['month_date', 'treated'], as_index=False).agg('mean')
count = msisdn_timeseries.groupby(['month_date', 'treated'], as_index=False).agg('count')
treated = avg[avg['treated'] == 1]
untreated = avg[avg['treated'] == 0]
ax.plot(treated['month_date'], 100*treated['msisdn_per_imei'], 
        label='Shared IMEIs', color='mediumseagreen')
ax.scatter(treated['month_date'], 100*treated['msisdn_per_imei'], color='mediumseagreen')
ax.plot(untreated['month_date'], 100*untreated['msisdn_per_imei'], color='mediumseagreen', dashes=[2, 2])
ax.scatter(untreated['month_date'], 100*untreated['msisdn_per_imei'], color='mediumseagreen')


ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%Y'))
ax.yaxis.set_major_formatter(mtick.PercentFormatter())

ax.set_title('Phone Sharing Over Time')
ax.legend(loc='best')
simpleaxis(ax)
plt.tight_layout()
plt.savefig('/home/em/phoneshare.png', dpi=300)
plt.show()

In [ ]:
outcome = 'imei_per_msisdn'
regressions = [
    ('Basic', ['treatmentxmonth', 'month_dummy_11', 'month_dummy_12', 'month_dummy_1']),
    ('Transactions Control', ['treatmentxmonth', 'msisdn_count_txns', 'month_dummy_11', 'month_dummy_12', 
                              'month_dummy_1']),
]

table = []
for regression, exog_vars in regressions:
    exog = sm.add_constant(data[exog_vars])
    mod = PanelOLS(data[outcome], exog, entity_effects=True, time_effects=False)
    result = mod.fit(cov_type='clustered', cluster_entity=True)
    
    stars = {result.pvalues.index[i]:'***' if result.pvalues[i] < 0.001 else '**' if result.pvalues[i] < 0.01 \
            else '*' if result.pvalues[i] < 0.05 else '' for i in range(len(result.pvalues))}
    params = {result.params.index[i]: ('%.4f' % result.params[i]) + stars[result.params.index[i]] \
              for i in range(len(result.params))}
    table.append({**params, **{'rsquared': '%.4f' % result.rsquared, 'n':'%i' % result.nobs}})
    
    
table = pd.DataFrame(table).T.sort_index().fillna('')
table.columns = [label for label, _ in regressions]
table = table.reindex(['const', 'treatmentxmonth', 'msisdn_count_txns', 'month_dummy_11', 'month_dummy_12', 
                       'month_dummy_1', 'rsquared', 'n'])
table

In [ ]:
outcome = 'msisdn_per_imei'
regressions = [
    ('Basic', ['treatmentxmonth', 'month_dummy_11', 'month_dummy_12', 'month_dummy_1']),
    ('Transactions Control', ['treatmentxmonth', 'imei_count_txns', 'month_dummy_11', 'month_dummy_12', 
                              'month_dummy_1']),
]

table = []
for regression, exog_vars in regressions:
    exog = sm.add_constant(data[exog_vars])
    mod = PanelOLS(data[outcome], exog, entity_effects=True, time_effects=False)
    result = mod.fit(cov_type='clustered', cluster_entity=True)
    
    stars = {result.pvalues.index[i]:'***' if result.pvalues[i] < 0.001 else '**' if result.pvalues[i] < 0.01 \
            else '*' if result.pvalues[i] < 0.05 else '' for i in range(len(result.pvalues))}
    params = {result.params.index[i]: ('%.4f' % result.params[i]) + stars[result.params.index[i]] \
              for i in range(len(result.params))}
    table.append({**params, **{'rsquared': '%.4f' % result.rsquared, 'n':'%i' % result.nobs}})
    
    
table = pd.DataFrame(table).T.sort_index().fillna('')
table.columns = [label for label, _ in regressions]
table = table.reindex(['const', 'treatmentxmonth', 'imei_count_txns', 'month_dummy_11', 'month_dummy_12', 
                       'month_dummy_1', 'rsquared', 'n'])
table